In [ ]:
import face_recognition
import cv2
import os
import numpy as np
import pandas as pd
from datetime import datetime

def load_student_encodings(images_path):
    encodings = []
    names = []
    for file in os.listdir(images_path):
        if file.endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(images_path, file)
            image = face_recognition.load_image_file(path)
            face_locations = face_recognition.face_locations(image)
            if len(face_locations) == 0:
                continue
            encoding = face_recognition.face_encodings(image, known_face_locations=face_locations)[0]
            encodings.append(encoding)
            name = os.path.splitext(file)[0]
            names.append(name)
    return encodings, names

def take_attendance(known_encodings, known_names):
    attendance = {}
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_encoding, face_location in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.5)
            face_distances = face_recognition.face_distance(known_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)

            if matches[best_match_index]:
                name = known_names[best_match_index]
                if name not in attendance:
                    attendance[name] = datetime.now().strftime("%H:%M:%S")
                top, right, bottom, left = face_location
                cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

        cv2.imshow("Attendance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    date = datetime.now().strftime("%Y-%m-%d")
    df = pd.DataFrame(list(attendance.items()), columns=["Name", "Time"])
    df.to_excel(f"attendance_{date}.xlsx", index=False)
    print(f"Saved in attendance_{date}.xlsx")

# الاستخدام
images_folder = "Students"  # set image folderض
encodings, names = load_student_encodings(images_folder)
take_attendance(encodings, names)
